In [27]:
import torch
import torchvision
import torch.nn as nn 
import torch.optim as optim  
from torch.utils.data import DataLoader

In [28]:
tensor_mode = torchvision.transforms.ToTensor()
trainset = torchvision.datasets.MNIST(root='./data', train=True, transform=tensor_mode, download=True)
testset = torchvision.datasets.MNIST(root='./data', train=False, transform=tensor_mode, download=True)
trainloader = DataLoader(dataset=trainset, batch_size=128, shuffle=True)
testloader = DataLoader(dataset=testset, batch_size=128, shuffle=False)

In [29]:
class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, seq_length, num_classes, device) -> None:
        super().__init__()
        self.device = device
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.seq_length = seq_length
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(seq_length*hidden_size*2, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(self.device)
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(self.device)
        out, _ = self.lstm(x, (h0, c0))
        out = out.reshape(-1, self.seq_length*self.hidden_size*2)
        out = self.fc(out)

        return out

In [30]:
trainset.data.shape

torch.Size([60000, 28, 28])

In [31]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sequence_length = trainset.data.size(1)
input_size = trainset.data.size(2)
num_layers = 2
hidden_size = 12
num_classes = 10

In [32]:
model = BiLSTM(input_size, hidden_size, num_layers, sequence_length, num_classes, device)
model = model.to(device)

In [33]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-3)

In [34]:
sample = next(iter(trainloader))

sample[0].shape, sample[0].squeeze().shape

(torch.Size([128, 1, 28, 28]), torch.Size([128, 28, 28]))

In [35]:
model.train()

for epoch in range(11):
    correct = 0
    total = 0
    for data in trainloader:
        images, labels = data[0].to(device).squeeze(1), data[1].to(device)
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        correct +=(output.argmax(axis=1) == labels).sum().item()
        total += len(labels)

    print(f'[{epoch}] train acc: {100*correct/total:.1f}')

[0] train acc: 91.44
[1] train acc: 97.59
[2] train acc: 98.16333333333333
[3] train acc: 98.495
[4] train acc: 98.735
[5] train acc: 98.79333333333334
[6] train acc: 99.00666666666666
[7] train acc: 99.11833333333334
[8] train acc: 99.095
[9] train acc: 99.25
[10] train acc: 99.27166666666666


In [37]:
def accuracy(dataloader):
    model.eval()

    correct = 0
    total = 0
    with torch.inference_mode():
        for data in dataloader:
            images, labels = data[0].to(device).squeeze(), data[1].to(device)
            outputs = model(images)
            correct += (outputs.argmax(axis=1) == labels).sum().item()
            total += len(labels)

        acc = 100*correct/total

    return acc

In [38]:
train_acc = accuracy(trainloader)
test_acc = accuracy(testloader)

print(f'Train Acc: {train_acc:.1f}, Test Acc: {test_acc:.1f}')

Train Acc: 99.6, Test Acc: 98.8
